In [6]:
from langgraph.graph import START, END, StateGraph
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver
from dotenv import load_dotenv
from typing import TypedDict
import os

In [4]:
load_dotenv()
LLM = ChatOpenAI(
    model="xiaomi/mimo-v2-flash:free",
    api_key=os.environ.get('OPEN_ROUTER_API_KEY'),
    base_url="https://openrouter.ai/api/v1",
    )

In [7]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str



In [11]:
def generate_joke(state):
    prompt = f'generate a joke on the topic {state["topic"]}'
    response = LLM.invoke(prompt).content

    return {'joke': response}


In [10]:
def generate_explanation(state):
    prompt = f'write an explanation for the joke {state["joke"]}'
    response = LLM.invoke(prompt).content

    return {'explanation': response}

In [12]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()
workflow = graph.compile(checkpointer=checkpointer)

In [17]:
config1 = {'configurable':{"thread_id":"1"}}
workflow.invoke({'topic':'pakistan economy'}, config=config1)

{'topic': 'pakistan economy',
 'joke': 'Here are a few options, ranging from light-hearted to a bit darker:\n\n**Option 1 (The International Comparison)**\nWhy are economists so excited about the Pakistan economy?\nBecause it’s the only place on earth where you can experience the thrill of a rollercoaster without ever leaving your house—the only difference is, in a rollercoaster, the drop eventually stops.\n\n**Option 2 (The Inflation Joke)**\nI asked my friend in Pakistan how he’s surviving the inflation.\nHe said, "It’s simple. I just pretend I\'m not hungry."\nI said, "That must be really hard."\nHe replied, "Actually, it’s getting easier every day."\n\n**Option 3 (The Short One-Liner)**\nI was going to make a joke about the Pakistani economy, but I can\'t afford to right now. The exchange rate is killing me.\n\n**Option 4 (The "Nuclear" Angle)**\nWe have two things that define us: We are a nuclear power, and we are an economic disaster.\nBasically, we have the power to blow up the 

In [25]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Here are a few options, ranging from light-hearted to a bit darker:\n\n**Option 1 (The International Comparison)**\nWhy are economists so excited about the Pakistan economy?\nBecause it’s the only place on earth where you can experience the thrill of a rollercoaster without ever leaving your house—the only difference is, in a rollercoaster, the drop eventually stops.\n\n**Option 2 (The Inflation Joke)**\nI asked my friend in Pakistan how he’s surviving the inflation.\nHe said, "It’s simple. I just pretend I\'m not hungry."\nI said, "That must be really hard."\nHe replied, "Actually, it’s getting easier every day."\n\n**Option 3 (The Short One-Liner)**\nI was going to make a joke about the Pakistani economy, but I can\'t afford to right now. The exchange rate is killing me.\n\n**Option 4 (The "Nuclear" Angle)**\nWe have two things that define us: We are a nuclear power, and we are an economic disaster.\nBasically, we have the power to b

In [20]:
workflow.get_state({'configurable': {'thread_id': '1','checkpoint_id': '1f0e98be-62b2-6eef-8002-c12c7aa354d0'}})

StateSnapshot(values={'topic': 'pakistan', 'joke': "Why don't Pakistanis ever get lost in the desert?\n\nBecause even if they run out of water, they can follow the trail of empty *chai* cups", 'explanation': 'Here is an explanation of the joke:\n\n**The Joke:**\n"Why don\'t Pakistanis ever get lost in the desert? Because even if they run out of water, they can follow the trail of empty *chai* cups."\n\n**The Explanation:**\n\nTo understand this joke, you need to understand three things: the setting, the pun, and the cultural reference.\n\n**1. The Pun (Wordplay)**\nThe joke relies on a play on the word **"Chai."**\n*   **Chai:** In South Asia (including Pakistan), "Chai" is the word for tea. It is an essential part of daily life and social culture.\n*   **Chain:** This sounds almost exactly like "Chai." In Hindi/Urdu, the word "Chain" means **"peace," "comfort,"** or **"rest."**\n\n**2. The Cultural Reference**\nPakistan is famous for its tea culture. "Chai khana" (drinking tea) is a m

In [21]:
workflow.invoke(None, {'configurable': {'thread_id': '1','checkpoint_id': '1f0e98be-62b2-6eef-8002-c12c7aa354d0'}})

{'topic': 'pakistan',
 'joke': "Why don't Pakistanis ever get lost in the desert?\n\nBecause even if they run out of water, they can follow the trail of empty *chai* cups",
 'explanation': 'Here is an explanation of the joke:\n\n**The Joke:**\n"Why don\'t Pakistanis ever get lost in the desert? Because even if they run out of water, they can follow the trail of empty *chai* cups."\n\n**The Explanation:**\n\nTo understand this joke, you need to understand three things: the setting, the pun, and the cultural reference.\n\n**1. The Pun (Wordplay)**\nThe joke relies on a play on the word **"Chai."**\n*   **Chai:** In South Asia (including Pakistan), "Chai" is the word for tea. It is an essential part of daily life and social culture.\n*   **Chain:** This sounds almost exactly like "Chai." In Hindi/Urdu, the word "Chain" means **"peace," "comfort,"** or **"rest."**\n\n**2. The Cultural Reference**\nPakistan is famous for its tea culture. "Chai khana" (drinking tea) is a major social activit

In [24]:
workflow.update_state({'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id':   '1f0e98c4-9483-6bc5-8006-e6f6fe85d17d'}},{'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0e9973-d05f-6c3e-8007-606c53563571'}}